# Note:
I started working on an async generator. Id doesn't seem to be working fast (~5 times slower). If you have any suggstions, please commit because it can improve performance up to 4 times and take only 10 minutes to run instead of 40 (I have I5 4460).

In [1]:
import asyncio
from aiostream import stream

async def batch_worker(ratings, intake, batch_size=25):
    while 1:
        
        batch_bar = tqdm(total=len(users))
        b = []
        for user, df in ratings.groupby(level=0):
            batch_bar.update(1)
            size = max(len(df) - frame_size, 0)
            for idx in range(0, size):
                if np.random.rand() < intake:  # intake percents
                    continue
                user_ratings = df[idx:frame_size + idx + 1]
                user_ratings = user_ratings[["movieId", "rating"]].values

                chosen_movie = user_ratings[:, 0][-1] 
                chosen_movie = movies[chosen_movie] # action
                chosen_rating = user_ratings[:, 1][-1] # reward
                films_watched = user_ratings[:, 0][:-1] 
                watched_rating = user_ratings[:, 1][:-1] # state
                watched_infos = [movies[i] for i in films_watched] # state
                watched_infos = torch.stack(watched_infos)    
                # state action reward
                b.append([(watched_infos, watched_rating), chosen_movie, chosen_rating])
                
                if len(b) >= batch_size:
                    yield b
                    b = []
        batch_bar.close()
        
async def batch_pool(n_workers, intake, batch_size):
    m_batch_size = batch_size/n_workers
    combine = stream.merge(*[batch_worker(ratings, intake, m_batch_size) for i in range(n_workers)])
    batch = []
    async with combine.stream() as streamer:
        async for m_batch in streamer:
            batch.append(m_batch)
            # cat all things, process batch
            if len(batch) >= n_workers:
                
                batch_copy = []
                
                for b in range(int(m_batch_size)):
                    for i in range(n_workers):
                        batch_copy.append(batch[i][b])
                batch = batch_copy
                del batch_copy
                
                yield (watched_infos, watched_rating), chosen_movie, chosen_rating
                batch = []

In [ ]:
async def main():
    i = 0
    async for batch in batch_pool(4, 0.2, 100):
        pass
loop = asyncio.get_event_loop()
asyncio.run_coroutine_threadsafe(main(), loop)